
<h1><center>Insurance Prediction. Data analysis and modeling.</center></h1>

<center><img src="https://www.outlookindia.com/outlookmoney/public/uploads/article/gallery/9f5518fc9b70672aaba65aa3af600c32.jpg"></center>


<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0; color:white' role="tab" aria-controls="home"><center>Quick navigation</center></h2>

* [1. Basic Data Analysis](#1)
* [2. Feature engineering (In progress)](#2)
* [3. Modeling (In progress)](#3)

<a id="1"></a>
<h2 style='background:blue; border:0; color:white'><center>1. Basic Data Analysis</center><h2>

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

import optuna
from optuna.samplers import TPESampler

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

!pip install pyod

from pyod.models.copod import COPOD

In [ ]:
train = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv')
test = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/test.csv')

#### train

In [ ]:
train = train.drop(['id'], axis=1)
test = test.drop(['id'], axis=1)

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Bar(
        x=['Male', 'Female'], 
        y=[
            len(train[train['Gender']=='Male']),
            len(train[train['Gender']=='Female'])
        ], 
        name='Train Gender',
        text = [
            str(round(100 * len(train[train['Gender']=='Male']) / len(train), 2)) + '%',
            str(round(100 * len(train[train['Gender']=='Female']) / len(train), 2)) + '%'
        ],
        textposition='auto'
    ),
    go.Bar(
        x=['Male', 'Female'], 
        y=[
            len(test[test['Gender']=='Male']),
            len(test[test['Gender']=='Female'])
        ], 
        name='Test Gender',
        text=[
            str(round(100 * len(test[test['Gender']=='Male']) / len(test), 2)) + '%',
            str(round(100 * len(test[test['Gender']=='Female']) / len(test), 2)) + '%'
        ],
        textposition='auto'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train/test gender column',
    height=400,
    width=700
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Bar(
        x=['Yes', 'No'], 
        y=[
            len(train[train['Driving_License']==1]),
            len(train[train['Driving_License']==0])
        ], 
        name='Train Driving_License',
        text = [
            str(round(100 * len(train[train['Driving_License']==1]) / len(train), 2)) + '%',
            str(round(100 * len(train[train['Driving_License']==0]) / len(train), 2)) + '%'
        ],
        textposition='auto'
    ),
    go.Bar(
        x=['Yes', 'No'], 
        y=[
            len(test[test['Driving_License']==1]),
            len(test[test['Driving_License']==0])
        ], 
        name='Test Driving_License',
        text=[
            str(round(100 * len(test[test['Driving_License']==1]) / len(test), 2)) + '%',
            str(round(100 * len(test[test['Driving_License']==0]) / len(test), 2)) + '%'
        ],
        textposition='auto'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train/test Driving_License column',
    height=400,
    width=700
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Bar(
        x=['Yes', 'No'], 
        y=[
            len(train[train['Previously_Insured']==1]),
            len(train[train['Previously_Insured']==0])
        ], 
        name='Train Previously_Insured',
        text = [
            str(round(100 * len(train[train['Previously_Insured']==1]) / len(train), 2)) + '%',
            str(round(100 * len(train[train['Previously_Insured']==0]) / len(train), 2)) + '%'
        ],
        textposition='auto'
    ),
    go.Bar(
        x=['Yes', 'No'], 
        y=[
            len(test[test['Previously_Insured']==1]),
            len(test[test['Previously_Insured']==0])
        ], 
        name='Test Previously_Insured',
        text = [
            str(round(100 * len(test[test['Previously_Insured']==1]) / len(test), 2)) + '%',
            str(round(100 * len(test[test['Previously_Insured']==0]) / len(test), 2)) + '%'
        ],
        textposition='auto'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train/test Previously_Insured column',
    height=400,
    width=700
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Bar(
        x=['Yes', 'No'], 
        y=[
            len(train[train['Vehicle_Damage']=='Yes']),
            len(train[train['Vehicle_Damage']=='No'])
        ], 
        name='Train Vehicle_Damage',
        text = [
            str(round(100 * len(train[train['Vehicle_Damage']=='Yes']) / len(train), 2)) + '%',
            str(round(100 * len(train[train['Vehicle_Damage']=='No']) / len(train), 2)) + '%'
        ],
        textposition='auto'
    ),
    go.Bar(
        x=['Yes', 'No'], 
        y=[
            len(test[test['Vehicle_Damage']=='Yes']),
            len(test[test['Vehicle_Damage']=='No'])
        ], 
        name='Test Vehicle_Damage',
        text = [
            str(round(100 * len(test[test['Vehicle_Damage']=='Yes']) / len(test), 2)) + '%',
            str(round(100 * len(test[test['Vehicle_Damage']=='No']) / len(test), 2)) + '%'
        ],
        textposition='auto'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train/test Vehicle_Damage column',
    height=400,
    width=700
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Bar(
        x=['> 2 Years', '1-2 Year', '< 1 Year'], 
        y=[
            len(train[train['Vehicle_Age']=='> 2 Years']),
            len(train[train['Vehicle_Age']=='1-2 Year']),
            len(train[train['Vehicle_Age']=='< 1 Year'])
        ], 
        name='Train Vehicle_Age',
        text = [
            str(round(100 * len(train[train['Vehicle_Age']=='> 2 Years']) / len(train), 2)) + '%',
            str(round(100 * len(train[train['Vehicle_Age']=='1-2 Year']) / len(train), 2)) + '%',
            str(round(100 * len(train[train['Vehicle_Age']=='< 1 Year']) / len(train), 2)) + '%'
        ],
        textposition='auto'
    ),
    go.Bar(
        x=['> 2 Years', '1-2 Year', '< 1 Year'], 
        y=[
            len(test[test['Vehicle_Age']=='> 2 Years']),
            len(test[test['Vehicle_Age']=='1-2 Year']),
            len(test[test['Vehicle_Age']=='< 1 Year'])
        ], 
        name='Test Vehicle_Age',
        text = [
            str(round(100 * len(test[test['Vehicle_Age']=='> 2 Years']) / len(test), 2)) + '%',
            str(round(100 * len(test[test['Vehicle_Age']=='1-2 Year']) / len(test), 2)) + '%',
            str(round(100 * len(test[test['Vehicle_Age']=='< 1 Year']) / len(test), 2)) + '%'
        ],
        textposition='auto'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train/test Vehicle_Age column',
    height=400,
    width=700
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Histogram(
        x=train['Age'], 
        name='Train Age'
    ),
    go.Histogram(
        x=test['Age'], 
        name='Test Age'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train/test Age column distribution',
    height=500,
    width=900
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Histogram(
        x=train['Annual_Premium'], 
        name='Train Annual_Premium'
    ),
    go.Histogram(
        x=test['Annual_Premium'], 
        name='Test Annual_Premium'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train/test Annual_Premium column distribution',
    height=500,
    width=800
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Histogram(
        x=train['Policy_Sales_Channel'], 
        name='Train Policy_Sales_Channel'
    ),
    go.Histogram(
        x=test['Policy_Sales_Channel'], 
        name='Test Policy_Sales_Channel'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train/test Policy_Sales_Channel column distribution',
    height=500,
    width=800
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Histogram(
        x=train['Vintage'], 
        name='Train Vintage'
    ),
    go.Histogram(
        x=test['Vintage'], 
        name='Test Vintage'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train/test Vintage column distribution',
    height=500,
    width=800
)
fig.show()

In [ ]:
tr = train['Region_Code'].value_counts().reset_index()
x_tr = tr['index'].tolist()
y_tr = tr['Region_Code'].tolist()
te = test['Region_Code'].value_counts().reset_index()
x_te = te['index'].tolist()
y_te = te['Region_Code'].tolist()

fig = make_subplots(rows=2, cols=1)

traces = [
    go.Bar(
        x=x_tr, 
        y=y_tr, 
        name='Train Region_Code'
    ),
    go.Bar(
        x=x_te, 
        y=y_te, 
        name='Test Region_Code'
    )
]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 1) + 1, (i % 1)  +1)

fig.update_layout(
    title_text='Train / test Region_Code',
    height=900,
    width=800
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=1)

traces = [
    go.Bar(
        x=['Yes', 'No'], 
        y=[
            len(train[train['Response']==1]),
            len(train[train['Response']==0])
        ], 
        name='Train Response'
    ),
]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train Response column',
    height=400,
    width=400
)
fig.show()

### As we can see from initial analysis all columns presented in dataset have exactly the same ditribution. Let's do feature engineering and modeling next.

In [ ]:
fig = px.histogram(
    train, 
    "Age", 
    color='Response',
    nbins=100, 
    title='Age & Response ditribution', 
    width=700,
    height=500
)
fig.show()

In [ ]:
fig = px.histogram(
    train, 
    "Age", 
    color='Response',
    nbins=100, 
    title='Age & Response ditribution', 
    width=700,
    height=500
)
fig.show()

In [ ]:
fig = px.histogram(
    train[train['Response'] == 1], 
    "Age", 
    nbins=100, 
    title='Age distribution for positive response', 
    width=700,
    height=500
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Bar(
        x=['Declined', 'Accepted'], 
        y=[
            len(train[(train['Gender']=='Male') & (train['Response']==0)]),
            len(train[(train['Gender']=='Male') & (train['Response']==1)])
        ], 
        name='Gender: Male'
    ),
    go.Bar(
        x=['Declined', 'Accepted'],  
        y=[
            len(train[(train['Gender']=='Female') & (train['Response']==0)]),
            len(train[(train['Gender']=='Female') & (train['Response']==1)])
        ], 
        name='Gender: Female'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train gender/response dependencies',
    height=400,
    width=700
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Bar(
        x=['Declined', 'Accepted'], 
        y=[
            len(train[(train['Previously_Insured']==0) & (train['Response']==0)]),
            len(train[(train['Previously_Insured']==0) & (train['Response']==1)])
        ], 
        name='Previously_Insured: Previously Not Insured'
    ),
    go.Bar(
        x=['Declined', 'Accepted'],  
        y=[
            len(train[(train['Previously_Insured']==1) & (train['Response']==0)]),
            len(train[(train['Previously_Insured']==1) & (train['Response']==1)])
        ], 
        name='Previously_Insured: Previously Insured'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train previously_insured/response dependencies',
    height=400,
    width=700
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)

traces = [
    go.Bar(
        x=['Declined', 'Accepted'], 
        y=[
            len(train[(train['Vehicle_Damage']=='No') & (train['Response']==0)]),
            len(train[(train['Vehicle_Damage']=='No') & (train['Response']==1)])
        ], 
        name='Vehicle_Damage: No'
    ),
    go.Bar(
        x=['Declined', 'Accepted'],  
        y=[
            len(train[(train['Vehicle_Damage']=='Yes') & (train['Response']==0)]),
            len(train[(train['Vehicle_Damage']=='Yes') & (train['Response']==1)])
        ], 
        name='Vehicle_Damage: Yes'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Train vehicle_damage/response dependencies',
    height=400,
    width=700
)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=3)

traces = [
    go.Bar(
        x=['Declined', 'Accepted'], 
        y=[
            len(train[(train['Vehicle_Age']=='> 2 Years') & (train['Response']==0)]),
            len(train[(train['Vehicle_Age']=='> 2 Years') & (train['Response']==1)])
        ], 
        name='Vehicle_Age: > 2 Years'
    ),
    go.Bar(
        x=['Declined', 'Accepted'], 
        y=[
            len(train[(train['Vehicle_Age']=='1-2 Year') & (train['Response']==0)]),
            len(train[(train['Vehicle_Age']=='1-2 Year') & (train['Response']==1)])
        ], 
        name='Vehicle_Age: 1-2 Year'
    ),
    go.Bar(
        x=['Declined', 'Accepted'], 
        y=[
            len(train[(train['Vehicle_Age']=='< 1 Year') & (train['Response']==0)]),
            len(train[(train['Vehicle_Age']=='< 1 Year') & (train['Response']==1)])
        ], 
        name='Vehicle_Age: < 1 Year'
    ),

]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 3) + 1, (i % 3)  +1)

fig.update_layout(
    title_text='Train/test Vehicle_Age/Response dependencies',
    height=400,
    width=800
)
fig.show()

In [ ]:
fig = px.histogram(
    train, 
    "Annual_Premium", 
    color='Response',
    nbins=100, 
    title='Annual_Premium & Response ditribution', 
    width=700,
    height=500
)
fig.show()

In [ ]:
fig = px.histogram(
    train[train['Response'] == 1], 
    "Annual_Premium", 
    nbins=100, 
    title='Annual_Premium distribution for positive response', 
    width=700,
    height=500
)
fig.show()

In [ ]:
fig = px.histogram(
    train, 
    "Vintage", 
    color='Response',
    nbins=100, 
    title='Vintage & Response ditribution', 
    width=700,
    height=500
)
fig.show()

In [ ]:
fig = px.histogram(
    train[train['Response'] == 1], 
    "Vintage", 
    nbins=100, 
    title='Vintage distribution for positive response', 
    width=700,
    height=500
)
fig.show()

<a id="2"></a>
<h2 style='background:blue; border:0; color:white'><center>2. Feature Engineering</center><h2>

#### 1) Convert columns with text values

In [ ]:
train.loc[train['Gender'] == 'Male', 'Gender'] = 1
train.loc[train['Gender'] == 'Female', 'Gender'] = 0
test.loc[test['Gender'] == 'Male', 'Gender'] = 1
test.loc[test['Gender'] == 'Female', 'Gender'] = 0

train.loc[train['Vehicle_Age'] == '> 2 Years', 'Vehicle_Age'] = 2
train.loc[train['Vehicle_Age'] == '1-2 Year', 'Vehicle_Age'] = 1
train.loc[train['Vehicle_Age'] == '< 1 Year', 'Vehicle_Age'] = 0
test.loc[test['Vehicle_Age'] == '> 2 Years', 'Vehicle_Age'] = 2
test.loc[test['Vehicle_Age'] == '1-2 Year', 'Vehicle_Age'] = 1
test.loc[test['Vehicle_Age'] == '< 1 Year', 'Vehicle_Age'] = 0

train.loc[train['Vehicle_Damage'] == 'Yes', 'Vehicle_Damage'] = 1
train.loc[train['Vehicle_Damage'] == 'No', 'Vehicle_Damage'] = 0
test.loc[test['Vehicle_Damage'] == 'Yes', 'Vehicle_Damage'] = 1
test.loc[test['Vehicle_Damage'] == 'No', 'Vehicle_Damage'] = 0

In [ ]:
for col in train.columns:
    train[col] = train[col].astype(np.int32)

train

In [ ]:
f = plt.figure(figsize=(13, 11))
plt.matshow(train.corr(), fignum=f.number)
plt.xticks(range(train.shape[1]), train.columns, fontsize=14, rotation=75)
plt.yticks(range(train.shape[1]), train.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)

### Correlation for every feature with target

In [ ]:
for col in train.columns:
    if col == 'Response':
        continue
    print(col, train[col].corr(train['Response']))

In [ ]:
fig = px.scatter(
    train, 
    x="Annual_Premium", 
    y="Age", 
    color="Response",
    width=600,
    height=600,
    title='Annual_premium vs Age scatter'
)
fig.show()

<a id="3"></a>
<h2 style='background:blue; border:0; color:white'><center>3. Modeling</center><h2>

#### Let's try unsupervised learning first. We will us kmeans clustering algorithm to check scores.

In [ ]:
X = train.drop(['Response'], axis=1)
y = train['Response']

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=666).fit(X)

In [ ]:
train['cluster'] = kmeans.labels_
train

In [ ]:
train['cluster'].value_counts()

In [ ]:
print('Kmeans accuracy: ', accuracy_score(train['Response'], train['cluster']))
print('Kmeans f1_score: ', f1_score(train['Response'], train['cluster']))

#### Now let's try to use COPOD anomaly detection model and check results

In [ ]:
response = train['Response']
train = train.drop(['Response', 'cluster'], axis=1)

In [ ]:
clf = COPOD(contamination=0.15)
clf.fit(train)

In [ ]:
cluster = clf.predict(train)
train['cluster'] = cluster
train['Response'] = response
train

In [ ]:
train['cluster'].value_counts()

In [ ]:
print('COPOD accuracy: ', accuracy_score(train['Response'], train['cluster']))
print('COPOD f1_score: ', f1_score(train['Response'], train['cluster']))

### Let's build our first version of classifier and use Logistic Regression

### Now we will create validation set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [ ]:
print('Positive cases % in validation set: ', round(100 * len(y_test[y_test == 1]) / len(y_test), 3), '%')
print('Positive cases % in train set: ', round(100 * len(y_train[y_train == 1]) / len(y_train), 3), '%')

#### So we can see that our sets are well balanced by target column and we can use our validation set for testing.

In [ ]:
model = LogisticRegression(random_state=666)
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)
print('Simple Logistic Regression accuracy: ', accuracy_score(y_test, preds))
print('Simple Logistic Regression f1_score: ', f1_score(y_test, preds))

In [ ]:
cm = confusion_matrix(y_test, preds)

ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, fmt='g')

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')

In [ ]:
X_train = X_train.drop(['Region_Code', 'Vintage', 'Driving_License'], axis=1)
X_test = X_test.drop(['Region_Code', 'Vintage', 'Driving_License'], axis=1)

In [ ]:
model = LogisticRegression(random_state=666)
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)
print('Simple Logistic Regression accuracy: ', accuracy_score(y_test, preds))
print('Simple Logistic Regression f1_score: ', f1_score(y_test, preds))

In [ ]:
cm = confusion_matrix(y_test, preds)

ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, fmt='g')

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')

#### After removing some columns predictions become better but still not good.

#### Let's build LightGBM with default parameters

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
model = LGBMClassifier(random_state=666)
model.fit(X_train, y_train)

preds = model.predict(X_test)
print('Simple LGBM accuracy: ', accuracy_score(y_test, preds))
print('Simple LGBM Regression f1_score: ', f1_score(y_test, preds))

#### Still not good. Let's try to optimize hyperparameters. to save time we will run optimization for 5 rounds but will take parameters that I got from previous versions.

In [ ]:
def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 30)
    n_estimators = trial.suggest_int("n_estimators", 1, 500)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0000001, 1)
    num_leaves = trial.suggest_int("num_leaves", 2, 5000)
    min_child_samples = trial.suggest_int('min_child_samples', 3, 200)
    model = LGBMClassifier(
        learning_rate=learning_rate, 
        n_estimators=n_estimators, 
        max_depth=max_depth,
        num_leaves=num_leaves, 
        min_child_samples=min_child_samples,
        random_state=0
    )
    return model

sampler = TPESampler(seed=0)
def objective(trial):
    model = create_model(trial)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    score = f1_score(y_test, preds)
    return score

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=5)

#lgb_params = study.best_params
lgb_params = {'max_depth': 26, 'n_estimators': 271, 'learning_rate': 0.6545636603165619, 'num_leaves': 2145, 'min_child_samples': 14}
lgb_params['random_state'] = 0
lgb = LGBMClassifier(**lgb_params)
lgb.fit(X_train, y_train)
preds = lgb.predict(X_test)
print('Optimized LightGBM: ', accuracy_score(y_test, preds))
print('Optimized LightGBM f1-score', f1_score(y_test, preds))

In [ ]:
cm = confusion_matrix(y_test, preds)

ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, fmt='g')

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')

In [ ]:
from xgboost import XGBClassifier

def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 30)
    n_estimators = trial.suggest_int("n_estimators", 1, 500)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0000001, 1)
    gamma = trial.suggest_uniform('gamma', 0.0000001, 1)
    model = XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, gamma=gamma, random_state=0)
    return model

def objective(trial):
    model = create_model(trial)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    score = f1_score(y_test, preds)
    return score

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=1)

#xgb_params = study.best_params
xgb_params = {
    'max_depth': 30, 
    'n_estimators': 136, 
    'learning_rate': 0.9867989680498112, 
    'gamma': 0.044733094210744555
}
xgb_params['random_state'] = 0
xgb = XGBClassifier(**xgb_params)
xgb.fit(X_train, y_train)
preds = xgb.predict(X_test)
print('Optimized XGBClassifier accuracy: ', accuracy_score(y_test, preds))
print('Optimized XGBClassifier f1-score', f1_score(y_test, preds))

In [ ]:
cm = confusion_matrix(y_test, preds)

ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, fmt='g')

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')

In [ ]:
cm /cm.astype(np.float).sum(axis=1)

In [ ]:
from sklearn.metrics import precision_score, recall_score
print(precision_score(y_test, preds))
print(recall_score(y_test, preds))

# Work in Progress

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
my_preds = preds.copy()

In [ ]:
X_test.get(1)

In [ ]:
#cm = confusion_matrix(y_test, preds)

fn_rate = confusion_matrix(y_test, preds_df)[1][0]
print(fn_rate)

In [ ]:
X_test.query('Age < 40 and Age > 30')

In [ ]:
X_test.loc[(X_test['Age'] > 35) & (X_test['Age'] < 40) & (X_test['Previously_Insured'] == 1) ].size / X_test.size

In [ ]:
print(X_test.loc[(X_test['Age'] > 40) & (X_test['Age'] < 45) & (X_test['Vehicle_Age'] == 2) ].size / X_test.size * 100)
print(X_train.loc[(X_train['Age'] > 40) & (X_train['Age'] < 45) & (X_train['Vehicle_Age'] == 2) ].size / X_train.size * 100)

In [ ]:
X_test.loc[(X_test['Previously_Insured'] == 0) ].size / X_test.size

In [ ]:
np.argmax(preds_proba[1])

In [ ]:
preds_argmax = [np.argmax(a) for a in preds_proba]

In [ ]:
all(preds_argmax == preds)

In [ ]:
##################################################################

In [ ]:
# save original test data 
x_test_orig = X_test.copy()
y_test_orig = y_test.copy()
preds_orig = preds.copy()

In [ ]:
# Set to Test data
# X_test = x_test_orig
# y_test = y_test_orig
# preds = preds_orig

# Set to Train data
X_test = X_train
y_test = y_train
preds = xgb.predict(X_train)

In [ ]:
preds_df = pd.Series(preds)
preds_df.index = y_test.index.tolist()

In [ ]:
# 0. Data segments data generator
MIN_VAL = 0
MAX_VAL = 100
INTERVAL = 10

conditions = []
for i in range(MIN_VAL, MAX_VAL, INTERVAL):
    conditions.append((i, i+INTERVAL))
    
for condition in conditions:
    cur_filter = 'Age > %d and Age <= %d' % (condition[0], condition[1])
#     cur_filter = 'Annual_Premium > %d and Annual_Premium <= %d' % (condition[0], condition[1])
#     cur_filter = 'Annual_Premium > %d and Annual_Premium < %d and Age >30 and Age <=40' % (condition[0], condition[1])
    filtered_indexes = X_test.query(cur_filter).index
#     print("%d - %d" % (condition[0], condition[1]))
    print("Age %d - %d,%d,%.3f,%.3f" % (condition[0], condition[1], X_test.loc[filtered_indexes].shape[0], X_test.loc[filtered_indexes].shape[0] / X_test.shape[0] * 100, accuracy_score(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes]),))
#     print('Segment size:', X_test.loc[filtered_indexes].shape[0])
#     print('Representing %.2f%%' % (X_test.loc[filtered_indexes].shape[0] / X_test.shape[0] * 100,))
#     print('Segment accuracy: %.2f' % (accuracy_score(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes]),))

In [ ]:
# 0.1 Generate deep combinations segments
import itertools

interesting_segs = """Age > 20 and Age <= 25
Age > 25 and Age <= 30
Age > 40 and Age <= 45
Age > 45 and Age <= 50
Annual_Premium > 2000 and Annual_Premium <= 3000
Policy_Sales_Channel == 157
Policy_Sales_Channel == 156
Policy_Sales_Channel == 124
Policy_Sales_Channel == 26
Policy_Sales_Channel == 152
Previously_Insured == 0
Previously_Insured == 1
Vehicle_Age == 0
Vehicle_Age == 1
Vehicle_Damage == 0
Vehicle_Damage == 1""".splitlines()

combinations = itertools.combinations(interesting_segs, 4)
filters = [' and '.join(c) for c in combinations]
filters = [f for f in filters if \
           f.count('Age ') <= 2 and f.count('Annual') <= 2 \
         and f.count('Policy') <=1 and f.count('_Age') <= 1 and f.count('_Damage') <= 1]

In [ ]:
len(filters)

In [ ]:
for cur_filter in filters:
    filtered_indexes = X_test.query(cur_filter).index
    print("%s,%d,%.3f,%.3f" % (cur_filter, X_test.loc[filtered_indexes].shape[0], X_test.loc[filtered_indexes].shape[0] / X_test.shape[0] * 100, accuracy_score(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes]),))

In [ ]:
# 1. Filter data by segment
#condition = 'Annual_Premium > 2000 and Annual_Premium <= 3000'# and Age > 30 and Age <= 40' # and Vehicle_Age == 2' # Annual_Premium < 10000'
condition = 'Annual_Premium > 2000 and Annual_Premium <= 3000 and Policy_Sales_Channel == 157 and Vehicle_Damage == 1' #Age > 35 and Age <= 40'# and Age > 30 and Age <= 40' # and Vehicle_Age == 2' # Annual_Premium < 10000'

filtered_indexes = X_test.query(condition).index
# negation_indexes = X_test.loc[~X_test.index.isin(filtered_indexes)].index

In [ ]:
# 2. Show segment size and %
print('Segment size:', X_test.loc[filtered_indexes].shape[0])
print('Representing %.2f%%' % (X_test.loc[filtered_indexes].shape[0] / X_test.shape[0] * 100,))

In [ ]:
# 3. Show accuracy for segment vs overall accuracy
print('All data accuracy: %.3f' % (accuracy_score(y_test, preds_df),))
print('Segment accuracy: %.3f' % (accuracy_score(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes]),))
print('All data recall: %.3f' % (recall_score(y_test, preds_df),))
print('Segment recall: %.3f' % (recall_score(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes]),))

In [ ]:
# 3.5. Show prediction distribution for segment
fig = px.histogram(
    preds_df.loc[filtered_indexes], 
    color = (preds_df.loc[filtered_indexes] == y_test.loc[filtered_indexes]),
    nbins=100, 
    title='Segment prediction distribution', 
    width=700,
    height=500
)
fig.show()

In [ ]:
# 4. Show proba() distribution for segment
preds_proba = xgb.predict_proba(X_test)
preds_proba_df = pd.DataFrame(data=preds_proba, columns=[0,1])
preds_proba_df.index = y_test.index.tolist()

fig = px.histogram(
    preds_proba_df.loc[filtered_indexes],
    nbins=100, 
    title='Segment probabilities distribution', 
    width=700,
    height=500
)
fig.show()

In [ ]:
# 5. Change argmax to another logic
def calc_segment_pred(row):
    if row[1] > 0.27:
        return 1
    return 0

all_preds_modified_df = preds_proba_df.apply(calc_segment_pred, axis=1)

# Change predictions
final_modified_preds_df = preds_df.copy()
# print('Modified Segment accuracy: %.2f%%' % (accuracy_score(y_test.loc[filtered_indexes], final_modified_preds_df.loc[filtered_indexes]),))
final_modified_preds_df.loc[filtered_indexes] = all_preds_modified_df.loc[filtered_indexes]

In [ ]:
# 6. Measure Recall
print('All data recall: %.3f%%' % (recall_score(y_test, preds_df),))

original_segment_recall = recall_score(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes])
print('Original Segment recall: %.3f%%' % (original_segment_recall,))

modified_segment_recall = recall_score(y_test.loc[filtered_indexes], final_modified_preds_df.loc[filtered_indexes])
print('Modified Segment recall: %.3f%%' % (modified_segment_recall,))

print('Modified all data recall: %.3f%%' % (recall_score(y_test, final_modified_preds_df),))

In [ ]:
# 6. Measure Accuracy
print('All data accuracy: %.3f%%' % (accuracy_score(y_test, preds_df),))
print('Original Segment accuracy: %.3f%%' % (accuracy_score(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes]),))
print('Modified Segment accuracy: %.3f%%' % (accuracy_score(y_test.loc[filtered_indexes], final_modified_preds_df.loc[filtered_indexes]),))
print('Modified all data accuracy: %.3f%%' % (accuracy_score(y_test, final_modified_preds_df),))

In [ ]:
# 7. confusion matrix for segment
cm = confusion_matrix(y_test.loc[filtered_indexes], final_modified_preds_df.loc[filtered_indexes])

ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, fmt='g')

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')

In [ ]:
# 7. confusion matrix for original segment
cm_orig = confusion_matrix(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes])

ax= plt.subplot()
sns.heatmap(cm_orig, annot=True, ax = ax, fmt='g')

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')

In [ ]:
# 6.5 business metrics
# print('%d more customers would get calls and buy insurance' % ((modified_segment_recall - original_segment_recall) * X_test.loc[filtered_indexes].shape[0],))
print('%d more customers would get calls and buy insurance (%d now vs. %d before)' % (cm[1][1] - cm_orig[1][1], cm[1][1], cm_orig[1][1]))
print('%d more calls (%d now vs. %d before) would be wasted by people that would not buy' % (cm[0][1] - cm_orig[0][1], cm[1][1] + cm[0][1], cm_orig[1][1] + cm_orig[0][1]))

In [ ]:
#*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
# Function: Plotting Confusion Matrix
def _plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens):
    from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score, roc_curve, auc, confusion_matrix
    import itertools
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize = 14)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="black")

    plt.ylabel('True Class', fontsize = 14)
    plt.xlabel('Predicted Class', fontsize = 14)

    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tight_layout()

In [ ]:
#*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
# Function: Finding thresholds
def _threshold_finder(model, X, y_true):
    from scipy import interp
    from sklearn.preprocessing import scale
    from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, roc_curve, confusion_matrix, average_precision_score, precision_recall_curve
    from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, train_test_split
    from xgboost import XGBClassifier
    import itertools
    import glmnet
    """
    a function to find the optimal threshold for binary classification
    model: a trained model object (such as xgboost, glmnet, ...)
    X: the test set of features (pandas dataframe or numpy array)
    y_true: the true class labels (list or array of 0's and 1's)    
    """
    
    y_predict_proba = model.predict_proba(X)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_true, y_predict_proba)
    auc = roc_auc_score(y_true, y_predict_proba)
    precision, recall, thresholds2 = precision_recall_curve(y_true, y_predict_proba)
    
    class_names = [0, 1]
    youden_idx = np.argmax(np.abs(tpr - fpr))
    youden_threshold = thresholds[youden_idx]
    y_pred_youden = (y_predict_proba > youden_threshold).astype(int)
    cnf_matrix = confusion_matrix(y_true, y_pred_youden)
    np.set_printoptions(precision=2)
    
    f1 = []
    for i in range(len(precision)):
        f1.append(2 * (precision[i] * recall[i]) / (precision[i] + recall[i]))
        
    queue_rate = []
    for thr in thresholds2:
        queue_rate.append((y_predict_proba >= thr).mean()) 

    plt.figure(figsize = (10, 5))
    plt.subplot(1,2,1)
    plt.plot(fpr, tpr, color = "red", label = F"AUC = {auc:.3f}")
    plt.plot(fpr[youden_idx], tpr[youden_idx], marker = "o", color = "navy", ms =10, label =F"Youden Threshold = {youden_threshold:.2f}" )
    plt.axvline(x = fpr[youden_idx], ymin = fpr[youden_idx], ymax = tpr[youden_idx], color = "navy", ls = "--")
    plt.plot([0,1], [0,1] , color = "black", ls = "--")
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.xlabel('1 - Specificity' , fontsize=12)
    plt.ylabel('Sensitivity' , fontsize=12)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.legend( prop={'size':12} , loc = 4)

    plt.subplot(1,2,2)
    _plot_confusion_matrix(cnf_matrix, classes=class_names, normalize = False, cmap=plt.cm.Reds, title = F"Youden Threshold = {youden_threshold:.2f}\nAccuracy = {accuracy_score(y_true, y_pred_youden)*100:.2f}%")
    plt.show()
    
    plt.figure(figsize = (12, 5))
    plt.subplot(1,2,1)
    plt.plot(thresholds, 1-fpr, label = "1 - Specificity")
    plt.plot(thresholds, tpr, label = "Sensitivity")
    plt.xlabel("Threshold", fontsize = 12)
    plt.ylabel("Score", fontsize = 12)
    plt.legend(loc = 0)
    plt.xlim([0.025, thresholds[np.argmin(abs(tpr + fpr - 1))]+0.2])
    plt.axvline(thresholds[np.argmin(abs(tpr + fpr - 1))], color="k", ls = "--")
    plt.title(F"Threshold = {thresholds[np.argmin(abs(tpr + fpr - 1))]:.3f}", fontsize = 12)
    
    plt.subplot(1,2,2)
    plt.plot(thresholds2, precision[1:], label = "Precision")
    plt.plot(thresholds2, recall[1:], label = "Recall")
    plt.plot(thresholds2, f1[1:], label = "F1-Score")
    plt.plot(thresholds2, queue_rate, label = "Queue Rate")
    plt.legend(loc = 0)
    plt.xlim([0.025, thresholds2[np.argmin(abs(precision-recall))] + 0.2])
    plt.xlabel("Threshold", fontsize = 12)
    plt.ylabel("Score", fontsize = 12)
    plt.axvline(thresholds2[np.argmin(abs(precision-recall))], color="k", ls = "--")
    plt.title(label = F"Threshold = {thresholds2[np.argmin(abs(precision-recall))]:.3f}", fontsize = 12)
    plt.show()

In [ ]:
_threshold_finder(xgb, X_test.loc[filtered_indexes], y_test.loc[filtered_indexes])

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt

# y_test= # ground truth labels
# preds = # predicted probabilities generated by sklearn classifier
skplt.metrics.plot_roc(y_test.loc[filtered_indexes], preds_proba_df.loc[filtered_indexes])
plt.show()

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt

# y_test= # ground truth labels
# preds = # predicted probabilities generated by sklearn classifier
skplt.metrics.plot_roc(y_test.loc[filtered_indexes], final_modified_preds_df.loc[filtered_indexes])
plt.show()dd

In [ ]:
#######################################

In [ ]:
X_test.loc[preds_df != y_test].to_csv('failed_training_samples.csv')

In [ ]:
preds_df = pd.Series(preds)
preds_df.index = y_test.index.tolist()

In [ ]:
# Setup as pandas series
preds_argmax_df = pd.Series(preds_argmax)
preds_argmax_df.index = y_test.index.tolist()

In [ ]:
accuracy_score(y_test.loc[filtered_indexes], preds_argmax_df.loc[filtered_indexes])

In [ ]:
accuracy_score(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes])

In [ ]:
# print(accuracy_score(
#     y_test[:],
#     preds[:]))
#cm_test = confusion_matrix(y_test, preds)
#print(cm_test[1][0] / sum(cm_test[1]))

#age_threshold = 60

# condition = 'Age > 50 and Annual_Premium > 100000'


# print(len(filtered_indexes))
# print(len(negation_indexes))
# print(len(filtered_indexes) + len(negation_indexes) == len(X_test))
#X_test.loc[X_test.query(condition).index]

# print(accuracy_score(
#     y_test.loc[filtered_indexes],
#     preds_df.loc[filtered_indexes]))

# printing FN Rate

cm_test = confusion_matrix(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes])
#print(cm_test[1][0] / sum(cm_test[1]))

cm_test /cm_test.astype(np.float).sum(axis=1)


print('All Data acc:' + str(accuracy_score(y_test, preds)))
print('Data slice acc:' + str(accuracy_score(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes])))

#cm_test = confusion_matrix(y_test.loc[negation_indexes], preds_df.loc[negation_indexes])
#print(cm_test[1][0] / sum(cm_test[1]))

In [ ]:
print(accuracy_score(
    y_test.loc[filtered_indexes],
    preds_df.loc[filtered_indexes]))

In [ ]:
# Filtered on selected data segment
cm = confusion_matrix(y_test.loc[filtered_indexes], preds_df.loc[filtered_indexes])

ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, fmt='g')

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')

In [ ]:
# Filtered on rest of data
cm = confusion_matrix(y_test.loc[negation_indexes], preds_df.loc[negation_indexes])

ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, fmt='g')

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')

In [ ]:
type(y_test)

In [ ]:
new_preds = pd.DataFrame(preds)
new_preds.insert(0, 'id', X_test.index)
new_preds.set_index('id')
new_preds.reindex()
#new_preds = new_preds.drop(columns=['id'])
#new_preds = pd.Series(new_preds)
#new_preds = new_preds.T.squeeze()
new_preds.head()

In [ ]:
new_preds.loc[filtered_indexes]

In [ ]:
type(y_test)

In [ ]:
X_test.query('Age > 50')

In [ ]:
y_test[X_test.query('Age > 50').index]

In [ ]:
#my_preds[X_test['Gender'] == 1] = preds[X_test['Gender'] == 1]
my_preds[X_test['Gender'] == 1] = preds[X_test['Gender'] == 1]
print(accuracy_score(y_test, preds))
print(accuracy_score(y_test, my_preds))

In [ ]:
X_test.loc[X_test['Gender'] == 1].shape

In [ ]:
X_test.head()

In [ ]:
X_test.to_csv(r'C:\temp\x_test.csv')

In [ ]:
pd.DataFrame(preds).to_csv('preds.csv')

In [ ]:
pd.DataFrame(y_test).to_csv('y_true.csv')

In [ ]:
(X_train, y_train, X_test, y_test, preds)